In [27]:
%%html
<script>
    var toggleCode= function() {
        var ins = document.getElementsByClassName("CodeMirror");
        for (var i=0; i < ins.length; i++) {
            ins[i].style.display= ins[i].style.display === "none" ? "block": "none";
        }
    };
</script>
<button onclick=toggleCode()>Toggle Code</button>

# Type 1 Diabetes Correlation Factors

## Introduction
------------

In this scenario we use AURIN's openapi to access dataset and try to find correlation factors about Type 1 Diabetes in Melbourne.

>**Data:**

> - Datasets available online from AURIN's openapi.

> - 'shapefiles' and 'excel' file about Type 1 Diabetes patient number and the SA2/postcode aggregation type.


## Data collecting
--------
After applying for the authentation, sending request to access AURIN's openapi and searching datasets based on their subjects.

For Type 1 Diabetes(T1D) data, reading data from source file.
### Extract data from OpenAPI
To check all available datasets in AURIN, please visit: https://aurin.org.au/datasets-available-in-the-openapi/

In [28]:
#Requesting to get the capabilities of the metadata service in AURIN.
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import configparser
import urllib.request, urllib.error, urllib.parse
from collections import defaultdict
from dbfread import DBF
from lxml import etree

# Import authentication parameters from the config file
config = configparser.RawConfigParser()
config.read('openapi.cfg')

#username: student
#password: dj78dfGF

username=config.get('Auth', 'username')
password=config.get('Auth', 'password')

# Submit an authenticated request to the AURIN Open API
def openapi_request(url):

    # create an authenticated HTTP handler and submit URL
    password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_manager.add_password(None, url, username, password)
    auth_manager = urllib.request.HTTPBasicAuthHandler(password_manager)
    opener = urllib.request.build_opener(auth_manager)
    urllib.request.install_opener(opener)
    
    #user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    #headers = { 'User-Agent' : user_agent }
    req = urllib.request.Request(url)
    try:
        handler = urllib.request.urlopen(req)
        return handler.read()
    except urllib.error.HTTPError as err:
        if err.code == 404:
            #When requesting the metadata by url, the server limits the successful times, so when
            #meeting HTTPError 404, print the message and request again.
            print ('Trying to access with url...')
            return openapi_request(url)
        else:
            raise
        
        
# Get the capabilities of the metadata service
url = 'http://openapi.aurin.org.au/csw?request=GetCapabilities&service=CSW'
xml = openapi_request(url)
root = etree.fromstring(xml)
print ("------------------------------------------------------------------------")
print ("Get capabilities successfully.")
print ("------------------------------------------------------------------------")

Trying to access with url...
------------------------------------------------------------------------
Get capabilities successfully.
------------------------------------------------------------------------


In [29]:
def get_all_datasets():
    dataset_names = []#This is a list of dataset names we can use to access its features

    # Get a list of all available datasets and their licences
    url='http://openapi.aurin.org.au/csw?request=GetRecords&service=CSW&version=2.0.2&typeNames=csw:Record&elementSetName=full&resultType=results&constraintLanguage=CQL_TEXT&constraint_language_version=1.1.0&maxRecords=1000'
    xml = openapi_request(url)
    root = etree.fromstring(xml)
     
    subs = set()#save all subjects available to choose
    for dataset in root.findall(".//csw:Record", root.nsmap):
        for subject in dataset.findall(".//dc:subject",root.nsmap):
            if not subject.text.startswith('ANZ'):
                subs.add(subject.text)
    print ('Please choose subject you are interested in from:')
    print (subs)
    target_sub = input('I want to search:')
     
    for dataset in root.findall(".//csw:Record", root.nsmap):
        #Start searching relavant datasets
        flag = 0
        for subject in dataset.findall(".//dc:subject",root.nsmap):
            if subject.text == target_sub:
                flag = 1
                break
        #Printing
        if flag == 1:
            print('================ DATASET + SUBJECTS ================')
            name = dataset.find(".//dc:identifier",root.nsmap).text
            dataset_names.append(name)
            print (name)
            for subject in dataset.findall(".//dc:subject",root.nsmap):
                print (subject.text) 
                
    return dataset_names
#get all available dataset names
dataset_names = get_all_datasets()

Trying to access with url...
Please choose subject you are interested in from:
{'crime', 'Queensland', 'born overseas', 'occupation', 'centroid', 'farms', 'salary', 'irsad', 'picnic', 'urban design', 'personal income', 'school type', 'broadhectare', 'facility', 'floor space ratio', 'education', 'unpaid work', 'New South Wales', 'households', '2015', 'census', 'HOB', 'polygon', 'height of building', 'equine', 'future', 'person characteristics', 'state heritage register', 'advantage', 'bus', 'housing', 'walkability', 'Victoria', 'qualifications', 'deaths', 'LSZ', 'total dwelling count', 'point', 'vicmap', 'public', 'ownership', 'LAP', 'queensland', 'road', 'diversity', 'health', 'school location', 'school', 'ptv', 'curtilage', 'South Australia', 'internet', 'residential', 'School', 'land use', 'expenditure', 'heritage', 'allocation', 'parcel', 'gender', 'child', 'profiles', 'ier', 'Brisbane', 'national', 'cadastre', 'industry', 'businesses', 'human', 'population', 'statistical', 'busines

================ DATASET + SUBJECTS ================
aurin:datasource-AU_Govt_ABS-UoM_AURIN_DB_1_nrp_econ_aust_sa2
census
regional profile
nrp
economy
businesses
business entries
business exits
industry
labour force
youth engagement
personal income
wage
salary
occupation
rent
mortgage
building approvals


In [30]:
#Let user choose dataset.
if dataset_names == []:
    raise Exception('No valid dataset is selected.')
else:
    print ('We found datasets below:',dataset_names)
    i = input('Please select the index that you want:')
    name = dataset_names[int(i)]
    print ('You have selected:',name)

We found datasets below: ['aurin:datasource-AU_Govt_ABS-UoM_AURIN_DB_1_nrp_econ_aust_sa2']


You have selected: aurin:datasource-AU_Govt_ABS-UoM_AURIN_DB_1_nrp_econ_aust_sa2


In [31]:
# Query the attributes of the selected dataset
def getFeatures(dataset):
    url = 'http://openapi.aurin.org.au/wfs?request=DescribeFeatureType&service=WFS&version=1.1.0&typeName='+name
    
    xml = openapi_request(url)
    root = etree.fromstring(xml)
    property_name = ''
    property_dict = defaultdict(list)
    agglevel = '' #either sa2 or postcode
    keylist = [] #candidate key names
    for element in root.find(".//xsd:sequence", root.nsmap):
        property_name = element.get('name')
        property_dict[property_name] = None
        print ('Property_name is:'+property_name)
        if 'post' in property_name and 'code' in property_name:
            agglevel = 'postcode'
            keylist.append(property_name)
        elif 'sa2' in property_name and 'main' in property_name:
            agglevel = 'sa2'
            keylist.append(property_name)
    if keylist == []:
        raise Exception('Invalid dataset. No valid aggregation level feature is included.')
    print (keylist)
    keyname = input('Please input a valid keyname:')
    if agglevel == '':
        raise Exception('Error input!')
    return property_dict,keyname,agglevel

#property_dict: save all features' name as keys and the value of some features will be saved
#keyname: It is the special feature which records the aggregation level. e.g. 'sa2_main11','actual_postcode' 
#agg_level: 'sa2' or 'postcode'
property_dict,keyname,agg_level = getFeatures(name)

Trying to access with url...
Property_name is:gid
Property_name is:sa2_main11
Property_name is:sa2_name11
Property_name is:year_
Property_name is:cabee_2
Property_name is:cabee_3
Property_name is:cabee_4
Property_name is:cabee_5
Property_name is:cabee_7
Property_name is:cabee_8
Property_name is:cabee_9
Property_name is:cabee_10
Property_name is:cabee_12
Property_name is:cabee_13
Property_name is:cabee_14
Property_name is:cabee_15
Property_name is:cabee_17
Property_name is:cabee_18
Property_name is:cabee_19
Property_name is:cabee_20
Property_name is:cabee_21
Property_name is:cabee_22
Property_name is:cabee_23
Property_name is:cabee_24
Property_name is:cabee_25
Property_name is:cabee_26
Property_name is:cabee_27
Property_name is:cabee_28
Property_name is:cabee_29
Property_name is:cabee_30
Property_name is:cabee_31
Property_name is:cabee_32
Property_name is:cabee_33
Property_name is:cabee_34
Property_name is:cabee_35
Property_name is:cabee_36
Property_name is:cabee_37
Property_name is:lf_

In [32]:
#Change the decimal longitude and latitude to Web Mercator Format, make it easier for data visualization
import math
def toWebMercator(xLon, yLat):
    # Check if coordinate out of range for Latitude/Longitude
    if (abs(xLon) > 180) and (abs(yLat) > 90):
        return
 
    semimajorAxis = 6378137.0  # WGS84 spheriod semimajor axis
    east = xLon * 0.017453292519943295
    north = yLat * 0.017453292519943295
 
    northing = 3189068.5 * math.log((1.0 + math.sin(north)) / (1.0 - math.sin(north)))
    easting = semimajorAxis * east
 
    return [easting, northing]

### Define geometry data collecting methods. 
There are two ways to get geometry data. 
1. Getting coordinates for polygons from shapefile(ENDIA file) when agg_level is SA2
2. Collecting coordinates data directly from url when agg_level is postcode.

In [33]:
#get Geometry data from the shapefile about t2d patient.
import shapefile
from geojson import Polygon
%matplotlib inline
def getGeometry_sa2(filename):
    poly_dict = defaultdict(Polygon)#key: sa2_main11 value:Polygon
    
    sf = shapefile.Reader(filename)
    #print (sf.fields)
    #395 in total
    for shape in sf.shapeRecords():
        poly = []
        key = shape.record[1]
        #print (key)
        for longi,lati in shape.shape.points:
            webMer_xy = toWebMercator(longi, lati)
            poly.append(webMer_xy)
            
        poly = Polygon([poly])
        poly_dict[str(key)]=poly    
    return poly_dict


In [34]:
#get Geometry data from the shapefile about t2d patient.
import shapefile
from geojson import Polygon,Point
%matplotlib inline
def getGeometry_postcode(dataset_name,keyname):
    #Plotting shape. 0 means drawing Points; 1 means drawing Polygon
    geo_shape = -1
    #Prepare to print progress
    barLength = 40
    
    #This can get all data with all features at the same time
    url = 'http://openapi.aurin.org.au/wfs?request=GetFeature&service=WFS&version=2.0.0&TypeName='+dataset_name
    
    #print('================ VALUES FOR A FEATURE ================')
    print('Query URL: '+url)
    
    xml = openapi_request(url)
    root = etree.fromstring(xml)
    instance_plots = []
    count = 0 
    poly_list = []#has a list of polygon
    #poly_dict = defaultdict(list)
    poly_dict = defaultdict(Polygon)
    total_instance = len(root.findall('.//wfs:member',root.nsmap))#total number of instances
    print ('Number of instance is',total_instance)
    
    for member in root.findall('.//wfs:member',root.nsmap):
        count += 1
        #each instance's geometry data are saved in a list
        ins_geometry = []
        
        #Get key value by searching element tag.
        key = ''
        for keyitem in member.iterfind('.//aurin:'+keyname,root.nsmap):
            key = keyitem.text
        
        #Find Polygon information 'posList'
        for poslist in member.iterfind('.//gml:posList',root.nsmap):
            geo_shape = 1 #drawing polygons later
            
            poslist = [float(x) for x in poslist.text.split()]
            #limit the range of longitude value
            lati,longi=[],[]
            poly = []
            for index in range(0,len(poslist)-1,2):
                lati.append(poslist[index])
                longi.append(poslist[index+1])
                
                #Since we plot polygons based on map which requires Web Mercator format
                webMer_xy = toWebMercator(poslist[index+1], poslist[index])
                poly.append(webMer_xy)
                
            ins_geometry.append((lati,longi))
            
            poly = Polygon([poly])
            
            poly_dict[key] = poly
            
            break #only go into the inner loop for once
        
        #Find Point information 'pos'
        for pos in member.iterfind('.//gml:pos',root.nsmap):
            geo_shape = 0 # Drawing Points later
            
            pos = [float(x) for x in pos.text.split()]
            
            #limit the range of longitude value
            lati,longi=pos[0],pos[1]
            
            #Since we plot polygons/points based on map which requires Web Mercator format
            webMer_xy = toWebMercator(longi, lati)
            
            poly = Point(webMer_xy)
            poly_dict[key] = poly
            
            break #only go into the inner loop for once
        #Print progress bar
        if count%100==0:
            progress = float(count)/total_instance
            block = int(round(barLength*progress))
            text = "\rPercent: [{0}] {1}%".format( "="*block + " "*(barLength-block), progress*100)
            print (text) 
    #Print the progress bar with 100% 
    block = barLength
    text = "\rPercent: [{0}] {1}%".format( "="*block + " "*(barLength-block), 100)
    print (text)  
    print (count,len(poly_dict))
    return poly_dict,geo_shape


### Specify features we are interested in. 
To make the tool  more efficient and durable, we limit the number of selected feature within 3.

In [35]:
prop_list = property_dict.keys()
max_feature_num = 3
#let users select features they want to analysis
#Asumming the feature selected mush be integer or float.
def selectFeatures():
    
    print ('Total features we have now are:',prop_list)
    
    selected = input("I select:") 
    #sa2_main11 or postcode can be seen as the key item in choose, so user does not need to add it. 
    choose = set([keyname])
    for name in selected.split():
        if name in prop_list:
            choose.add(name)
            if len(choose)==max_feature_num:
                break       
    return choose

#choose: saving all selected features and the keyname(the feature which specifies aggregation level)
choose = selectFeatures()
print ('Selected features are:',choose)

Total features we have now are: dict_keys(['bldg_11', 'cabee_26', 'bldg_6', 'wageocc_17', 'bldg_3', 'year_', 'wageocc_10', 'wage_34', 'income_18', 'sa2_main11', 'wageocc_13', 'income_11', 'wage_35', 'cabee_12', 'wage_25', 'bldg_2', 'wageocc_7', 'cabee_35', 'wage_8', 'income_5', 'income_10', 'income_13', 'wage_6', 'cabee_36', 'cabee_25', 'wage_26', 'wageocc_4', 'wageocc_20', 'income_8', 'wage_15', 'lf_5', 'wage_9', 'cabee_8', 'youth_6', 'cabee_32', 'wage_33', 'income_4', 'cabee_2', 'wage_22', 'cabee_27', 'wage_7', 'wage_2', 'cabee_18', 'youth_8', 'shape_area', 'youth_3', 'cabee_15', 'cabee_10', 'youth_2', 'wageocc_18', 'bldg_5', 'income_6', 'wageocc_15', 'bldg_9', 'youth_9', 'wageocc_12', 'cabee_19', 'wage_11', 'wageocc_8', 'income_19', 'wage_32', 'wageocc_3', 'wage_19', 'wageocc_6', 'wage_23', 'cabee_22', 'cabee_34', 'sa2_name11', 'income_12', 'rent_2', 'cabee_20', 'cabee_24', 'wage_20', 'wage_18', 'bldg_10', 'bldg_8', 'wage_31', 'cabee_29', 'income_16', 'lf_4', 'wage_27', 'cabee_21', 

In [36]:
#Given a feature, getFeatureValue will return all values.
def getFeatureValue(prop):
    #This can get all values for property_name
    url = 'http://openapi.aurin.org.au/wfs?request=GetPropertyValue&service=WFS&version=2.0.0&TypeName='+name+'&valueReference='+prop
    #print (url)
    
    xml = openapi_request(url)
    root = etree.fromstring(xml)
    #i = 0
    values = []
    for member in root.findall('.//wfs:member',root.nsmap):
        value = member.find('.//aurin:'+prop,root.nsmap).text
        values.append(value) 
    return values

### Collecting geometry data. 
After selecting features needed to fetch values, we start collecting geometry data and the type.(Points or Polygons) 

In [37]:
geo_shape = -1
if agg_level == 'sa2':
    #key is sa2 code, value is Polygon objective.
    polygon_dict = getGeometry_sa2('endia-sa2/04eba726-cea8-4ba6-8911-0be82949c851.shp')
    geo_shape = 1 #Drawing polygons later
elif agg_level == 'postcode':
    polygon_dict,geo_shape = getGeometry_postcode(name,keyname)
    
print ("------------------------------------------------------------------------")
print ("Geometry data successfully collected")
print ("------------------------------------------------------------------------")

------------------------------------------------------------------------
Geometry data successfully collected
------------------------------------------------------------------------


### Collecting normal featuers' values.
*when seeing message 'HTTPError 404', please ignore because it will not affect the result.

In [38]:
#collect values for keyname
property_dict[keyname] = getFeatureValue(keyname)
instance_num = len(property_dict[keyname])
total_ins = instance_num
print ('Total instance number is %d and finish collecting property \"%s\"' %(instance_num,keyname))

#If a features values is not complete (smaller than total_ins), discard it.
new_choose = [keyname]
#collect values for normal features
for prop in prop_list:
    if prop in choose and prop != keyname:
        property_dict[prop] = getFeatureValue(prop)
        instance_num = len(property_dict[prop])
        if instance_num == total_ins:
            new_choose.append(prop)
            print ('Total instance number is %d and finish collecting property \"%s\"' %(instance_num,prop))
        else:
            print ('Property',prop,'has insufficient values, will remove it from choose')
choose = new_choose
print ("------------------------------------------------------------------------")
print ('Completed collected normal features values.')
print ('Updated candidate features are',choose)
print ("------------------------------------------------------------------------")

Trying to access with url...
Total instance number is 11021 and finish collecting property "sa2_main11"
Trying to access with url...
Total instance number is 11021 and finish collecting property "cabee_12"
Trying to access with url...
Total instance number is 11021 and finish collecting property "income_5"
------------------------------------------------------------------------
Completed collected normal features values.
Updated candidate features are ['sa2_main11', 'cabee_12', 'income_5']
------------------------------------------------------------------------


### Read T1D data from source data
-Read shapefile from ENDIA when aggregation level is SA2

-Read csv file from ADDN when aggregation level is postcode

In [39]:
#patient_dict is a dictionary which keeps keyname as key and patient_num as value.
import csv
patient_dict = defaultdict(int)

#Collect patient distribution at SA2 level.
if agg_level == 'sa2':
    for record in DBF('endia-sa2/04eba726-cea8-4ba6-8911-0be82949c851.dbf'):
        patient_dict[str(record['SA2_MAIN11'])] = int(record['count'])
    #print ('The number of different aggregation level areas is:',len(patient_dict.keys()))
#Collect patient distribution at postcode level.   
elif agg_level == 'postcode':
    print ('Please choose 0(diagnosed postcode); 1(current postcode)')
    index = input('I select')
    index = int(index)
    if index != 0 and index != 1:
        raise Exception('Input error!')
    with open('postcode_pairs.csv', 'r') as infile:
        r = csv.reader(infile)
        next(r)
        for row in r: 
            patient_dict[str(row[index])] = int(row[2])
    #print ('The number of different aggregation level areas is:',len(patient_dict.keys()))
print ("------------------------------------------------------------------------")
print ('Aggregation level code successfully collected.')
print ("------------------------------------------------------------------------")

------------------------------------------------------------------------
Aggregation level code successfully collected.
------------------------------------------------------------------------


### Comebine all data together. 
normal features' values<-->geometry data<-->patients' information

Using aggregation level(sa2/postcode) as bridges to save all relative information together

In [40]:
#dic:(sa2_code:dictionary). With all normal features' values.
def combineDict(dic,patient_dict,geom_dict):
    #Each item in instances is a dict with all information.
    #If common keyname's value occure, put all informatin together as one instance.
    instances = []
    for k,list_val in dic.items():
        if len(list_val) == 1:
            list_val[0].update({'geometry':geom_dict[k],'count':int(patient_dict[k])})
            instances.append(list_val[0])
        elif len(list_val)>1:
            basedict = list_val[0]
            basedict.update({'count':int(patient_dict[k])})
            basedict.update({'geometry':geom_dict[k]})
            
            for newdict in list_val[1:]:
                for prop in choose:
                    if 'geom' not in prop and prop != keyname:
                        #if values are countable, add them up, otherwise, ues commans to divide.
                        try:
                            basedict[prop] = float(basedict[prop])
                            newdict[prop] = float(newdict[prop])
                            basedict[prop] += newdict[prop]
                        except ValueError:
                            basedict[prop] += ','+newdict[prop]
            instances.append(basedict)
        else:
            "error!"
    return instances

In [41]:
from geojson import Feature,FeatureCollection,Polygon,GeometryCollection
import geojson
# property_dict save property_name as key, and a dictionary containing type and value as value.
# Now we need to change the format by saving a list of Instance objects which represent many instances(row).
def changeFormat(property_dict,candidates):
    instance_dict = defaultdict(list) #key=sa2_main11 value:dict
    for i in range(instance_num):
        #check whether the instance occur in the t2d file.
        if property_dict[keyname][i] not in patient_dict.keys():
            continue
        #print (property_dict['sa2_main11']['value'][i])
        #properties = dict() #save all values except geometry
        ins = dict()
        for k,v in property_dict.items():
            if k in candidates:
                if k == keyname:
                    #update attribute in instance
                    ins.update({k:str(v[i])})
                elif 'geom' not in k:
                    #update attribute in instance
                    ins.update({k:str(v[i])})
                    
        instance_dict[ins[keyname]].append(ins)
        
    instances = combineDict(instance_dict,patient_dict,polygon_dict) #except combing item with same sa2, but also add count of patient
    
    #add valid data into feature_list
    feature_list = []#for creating a featureCollection to plot later
    for instance in instances:
        properties = dict()
        for k,v in instance.items():
            if k != 'geometry':
                properties.update({k:v})
        #print (type(instance),type(instance['geometry']))
        feature = Feature(geometry=instance['geometry'], properties=properties)
        feature_list.append(feature)
        
    return instances,feature_list


instances,feature_list = changeFormat(property_dict,choose)  
print ("------------------------------------------------------------------------")
print ('Format change completed.')
print ("------------------------------------------------------------------------")

------------------------------------------------------------------------
Format change completed.
------------------------------------------------------------------------


## Visualization
--------
Based on the number of T1D patients in specific SA2 area, map the data to a Australia map and using different colors to represent differencet number of patients.

If the dataset is at postcode level, we draw blue points to represent different postcode areas. In this case, number of T1D patients cannot be reflected by color but can be seen in the HoverTool.


In [42]:
from geojson import FeatureCollection

fc = FeatureCollection(feature_list)
geojsonData = geojson.dumps(fc)

In [43]:
#Base map layer,showing AU map
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.tile_providers import WMTSTileSource

AU = x_range,y_range = ((11884029,17283304), (-688291,-4556972))

#fig = figure(tools='pan, wheel_zoom', x_range=x_range, y_range=y_range)
TOOLS="pan,wheel_zoom,box_zoom,hover,save"
fig = figure(title="Water and Energy consuming data", tools=TOOLS,x_range=x_range, y_range=y_range)

fig.axis.visible = False
url = 'http://a.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png'
attribution = "Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL"

fig.add_tile(WMTSTileSource(url=url, attribution=attribution))
output_notebook()
#show(fig)

Loading BokehJS ...

In [44]:
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool, LinearColorMapper, LogColorMapper
from bokeh.palettes import Purples6 as palettex

geo_source = GeoJSONDataSource(geojson=geojsonData)

palettex = palettex[::-1]
color_mapper = LogColorMapper(palette=palettex)

fig.title.text_font_size='12pt'
fig.title.align='center'

if geo_shape == 0:
    #Drawing circles when meeting Points
    fig.circle('x','y',size=4,color = 'blue',source=geo_source)
elif geo_shape == 1:
    #Drawing polygons when meeting Polygons
    fig.patches('xs','ys',fill_alpha=.99, fill_color={'field': 'count', 'transform': color_mapper}
              ,line_color="#884444", line_width=0.1, line_alpha=0.8,source=geo_source)

hover = fig.select_one(HoverTool)
hover.point_policy = "follow_mouse"
display_prop = [('count','@count')]
for prop in choose:
    if 'geom' not in prop:
        display_prop.append((prop,'@'+prop))
hover.tooltips = display_prop

#output_notebook()
show(fig)


## Analysis
--------
The main idea is to see if there is an association between number of patients and countable features that user selected from the dataset. To process discret data, Maximum Information Coefficient(MIC) is used.

If there is, we can do some deeper analysis to find a fine model which describe the relationship better.This tool offer three regression models: Linearregression, logicregression and multinomial naive bayes.

In [ ]:
#Since we convert all normal feature values types to string. In this part, for countable numbers,
# convert them back to float and using MIC to represent their correlation with patient number
from minepy import MINE

m = MINE()
#x records number of patients
x = []
for ins in instances:
    x.append(ins['count'])

#countable feature values
total_ins = len(instances)
ys = defaultdict(list) # save feature name and their values
for feature in choose:
    y = []
    for instance in instances:
        try:
            value = float(instance[feature])
            y.append(value)
        except ValueError:
            print ("%s are uncountable."% feature)
            break
    if len(y) == total_ins:
        ys[feature] = y
print ("------------------------------------------------------------------------")
print ('Countable values successfully collected.')
print ("------------------------------------------------------------------------")

### Visualize correlation between features.

In [ ]:
def drawLine(x,y,featurename):
    # create a new plot with default tools, using figure
    title = "Correlation bewteen number of patient and "+featurename
    p = figure(title = title,plot_width=500, plot_height=400)
    # add a circle renderer with a size, color, and alpha
    #p.line(x, y, line_width=2)
    p.circle(x, y, line_color="navy", fill_color="red", size=8)
    show(p) # show the results
    
for k,val in ys.items():  
    m.compute_score(x,val)
    drawLine(x,val,k)
    print('Maximal information coefficient between T1D patient number and %s is %f'% (k,m.mic()))

### Further analysis
If there is a close relationship between certain features and number of patients, we use three models to help analyse data. 
Before training data, DictVectorizer is used for creating a sparse matrix, in this case, more features will be created.

In [ ]:
#Build training data set and apply several models to see the model's score
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,LinearRegression

print (choose)
features = input('I select features:')
features = features.split()
#print (features)

feature_matrix = []#training data
data_label = []#training set class values
for instance in instances:
    dict1 = {}
    for feature in features:
        dict1.update({feature:instance[feature]})
    feature_matrix.append(dict1)
    data_label.append(instance['count'])
        
vectorizer = DictVectorizer()
data_dataset = vectorizer.fit_transform(feature_matrix).toarray()
#print (data_dataset.shape,len(data_label))

#divide dataset into train_dataset and dev_dataset
total = len(instances)
dev_num = total//3 #dev_dataset counts for 1/3 of total dataset

dev_dataset = data_dataset[:dev_num]
dev_classification = data_label[:dev_num]

train_dataset = data_dataset[dev_num:]
train_classification = data_label[dev_num:]

# Start traing with--------LinearRegression--------------------
try:
    ln = LinearRegression()
    ln.fit(train_dataset, train_classification)          
    acc_ln = ln.score(dev_dataset,dev_classification)    
    print ('Accuracy for model LinearRegression is',acc_ln)
except ValueError:
    print ('Current dataset has invalid values, cannot build Linear Regression model.')


# Start traing with--------LogisticRegression--------------------
try:
    lr = LogisticRegression()
    lr.fit(train_dataset, train_classification)          
    acc_lr = lr.score(dev_dataset,dev_classification)
    print ('Accuracy for model LogisticRegression is',acc_lr)
except ValueError:
    print ('Current dataset has invalid values, cannot build Logistic Regression model.')

# Start traing with--------Multinomial Naive Bayes--------------------
try:
    mnb = MultinomialNB()
    mnb.fit(train_dataset, train_classification)  
    acc_mnb = mnb.score(dev_dataset,dev_classification)
    print ('Accuracy for model MultinomialNB is',acc_mnb)
except ValueError:
    print ('Current dataset has invalid values, cannot build Multinomial Naive Bayes model.')
